# Prompting the models to classify the statements

In [28]:
import pandas as pd

embedding = "te3l" # / "te3s"
grobid_model = "full_model"
no_prev_chunking = False

path = f"../data/dfs/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/ReferenceErrorDetection_data_with_chunk_info.pkl"
print(path)

# read the dataframe from a pickle file
df = pd.read_pickle(path)

../data/dfs/te3l/full_model/ReferenceErrorDetection_data_with_chunk_info.pkl


In [29]:
df.head()

,Source,Citing Article ID,Citing Article DOI,Citing Article Title,Citing Article Retracted,Citing Article Downloaded,Domain,Statement with Citation,Reference Article ID,Reference Article DOI,Reference Article Title,Reference Article Abstract,Reference Article PDF Available,Reference Article Retracted,Reference Article Downloaded,Label,Explanation,Top_3_Chunk_IDs,Top_3_Chunk_Texts
0,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,Others have aimed to reduce irreversibility or...,r001,10.1155/2021/2087027,A Fault Analysis Method for Three-Phase Induct...,The fault prediction and abductive fault diagn...,Yes,No,Yes,Unsubstantiate,Irrelevant,"[ecb51e4f-6f0b-4abe-8fd1-0b1119d3c57b, 3543f03...",[implementation of fuzzy reasoning spiking neu...
1,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,Some researchers have also studied various hea...,r002,10.1016/j.physa.2018.12.031,Develop 24 dissimilar ANNs by suitable archite...,The artificial neural network optimization met...,Yes,No,Yes,Unsubstantiate,Irrelevant,"[7609e3bb-6a78-4479-b2fc-23986dba04bc, f25f6b4...",[Sci\n\t\t\n\t\t\t79\n\t\t\t\n\t\t\t2016\n\t\t...
2,PubPeer,c002,10.1155/2022/4601350,Oxidative Potential and Nanoantioxidant Activi...,Yes,Yes,Chemistry,The relative content of total flavonoids in th...,r003,10.1088/1742-6596/1937/1/012038,Lipid Data Acquisition for devices Treatment o...,"Recently, the widespread deployment of smart p...",Yes,No,Yes,Unsubstantiate,Irrelevant,"[19f64acb-c4e7-40b4-bb88-bf12bfd7fda1, 08e6ae8...",[AND RESULTPhotochemical blood performance of ...
3,PubPeer,c003,10.1155/2022/2408685,The Choice of Anesthetic Drugs in Outpatient H...,Yes,Yes,Medicine,Research has shown that remimazolam tosylate e...,r004,10.1186/s12871-018-0543-3,"Effect of propofol on breast cancer cell, the ...",Breast cancer is the second leading cause of c...,Yes,No,Yes,Unsubstantiate,Irrelevant,"[9dd7fb55-09ad-488c-a9c3-0a25a9cfa560, ebb664a...",[Rep\n\t\t\n\t\t\t37\n\t\t\t2\n\t\t\t\n\t\t\t2...
4,PubPeer,c004,10.1155/2022/4783847,A Fault-Tolerant Structure for Nano-Power Comm...,Yes,Yes,Engineering,if the efficiency of the routing algorithm is ...,r005,10.36410/jcpr.2022.23.3.312,Analysis and research hotspots of ceramic mate...,"From the perspective of scientometrics, comb t...",Yes,No,Yes,Unsubstantiate,Irrelevant,"[fb8d6cc8-5f22-4add-a5c6-4843d8d61043, 199ad27...",[with high centrality are usually the main nod...


## Create the prompts

In [30]:
def format_excerpts(excerpt_list):
    excerpts_text = ""
    for id, excerpt in enumerate(excerpt_list):
        excerpts_text += f"Excerpt {id+1}: \n{excerpt}\n\n"
    return excerpts_text

In [31]:
print(format_excerpts(df.iloc[0]['Top_3_Chunk_Texts']))

Excerpt 1: 
implementation of fuzzy reasoning spiking neural P systems for diagnosing faults in complex power systems
		
			HNRong
		
		
			KYi
		
		
			GXZhang
		
		
			JDong
		
		
			PPaul
		
		
			ZHuang
		
	
	
		Complexity
		
			2019
			16
			2019
		
	



	
		A rough set-based bioinspired fault diagnosis method for electrical substations
		
			TWang
		
		
			WLiu
		
		
			JBZhao
		
	
	
		International Journal of Electrical Power & Energy Systems
		
			119
			105961
			2020
		
	



	
		Simplified and yet turing universal spiking neural P systems with polarizations optimized by anti-spikes
		
			TWu
		
		
			TZhang
		
		
			FXu
		
	
	
		Neurocomputing
		
			414
			
			2020
		
	



	
		Spiking neural P systems with polarizations and rules on

Excerpt 2: 
Transactions on Transportation Electrification
		
			6
			3
			
			2020
		
	



	
		Detection of deterioration of three-phase induction motor using vibration signals
		
			AGlowacz
		
		
			WGlowacz
		
		
			JKozik
		
	
	
		Measuremen

In [32]:
def create_prompt(df_row):
    title = df_row['Citing Article Title']
    statement = df_row['Statement with Citation']
    reference_title = df_row['Reference Article Title']
    reference_abstract = df_row['Reference Article Abstract']
    reference_excerpts = format_excerpts(df_row['Top_3_Chunk_Texts'])

    prompt = f"""   
You are an experienced scientific writer and editor. 
You will be given a statement from an article that cites a reference article and information from the reference article. 
You will determine and explain if the reference article supports the statement.  
    
Specifically, choose a label from "Fully substantiate", "Partially substantiate", and "Unsubstantiate". 
Further explanations of the labels are as follows: 
"Fully substantiated": The reference article fully substantiates the relevant part of the statement from the present article. 
"Partially substantiated": According to the reference article, there is a minor error in the statement but the error does not invalidate the purpose of the statement. 
"Unsubstantiate": The reference part does not substantiate any part of the statement. This could be because the statement is contradictory to, unrelated to, or simply missing from the reference article.  
    
Format your answer in JSON with two elements: "label" and "explanation". 
Your explanation should be short and concise. 
    
# The citing article
Title: {title} 
Statement: {statement}
    
# The reference article 
Title: {reference_title} 
Abstract: {reference_abstract} 
Excerpts: \n\n{reference_excerpts}
"""

    return prompt

In [33]:
example_prompt = create_prompt(df.iloc[22])
print(example_prompt)

   
You are an experienced scientific writer and editor. 
You will be given a statement from an article that cites a reference article and information from the reference article. 
You will determine and explain if the reference article supports the statement.  
    
Specifically, choose a label from "Fully substantiate", "Partially substantiate", and "Unsubstantiate". 
Further explanations of the labels are as follows: 
"Fully substantiated": The reference article fully substantiates the relevant part of the statement from the present article. 
"Partially substantiated": According to the reference article, there is a minor error in the statement but the error does not invalidate the purpose of the statement. 
"Unsubstantiate": The reference part does not substantiate any part of the statement. This could be because the statement is contradictory to, unrelated to, or simply missing from the reference article.  
    
Format your answer in JSON with two elements: "label" and "explanation"

## Prompting the models (batch processing)

In [7]:
import os
import json

def create_batch_file(df, model="gpt-3.5-turbo-0125"):
    output_dir = f"../data/batch_files/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}"
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, "prompt_batch.jsonl")
    
    # If the file already exists, empty it
    if os.path.exists(output_file):
        open(output_file, "w").close()
    
    for index, row in df.iterrows():
        if row['Reference Article Downloaded'] == 'Yes':
            prompt = create_prompt(row)
            json_sequence = {
                "custom_id": f"request-{index}", 
                "method": "POST", 
                "url": "/v1/chat/completions", 
                "body": {
                    "model": model, 
                    "messages": [
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ],
                    "temperature": 0,
                }
            }

            with open(output_file, "a") as f:
                f.write(json.dumps(json_sequence) + "\n")
                
    return output_file

In [8]:
batch_file_path = create_batch_file(df)
batch_file_path

'../data/batch_files/te3l_no_prev_chunking/full_model/prompt_batch.jsonl'

In [9]:
# Read the content of open_ai_key.txt into a variable
with open('../open_ai_key.txt', 'r') as file:
    open_ai_key = file.read().strip()

In [10]:
from openai import OpenAI
client = OpenAI(api_key=open_ai_key)

batch_input_file = client.files.create(
    file=open(batch_file_path, "rb"),
    purpose="batch"
)

print(batch_input_file)

FileObject(id='file-Q2nb2DDM5idxjmsQMK4v9L', bytes=1474919, created_at=1742833919, filename='prompt_batch.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)


In [11]:
batch_input_file_id = batch_input_file.id
batch_creation_response = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h"
)

In [12]:
batch_creation_response

Batch(id='batch_67e189022f3c8190be971477ff6a73bc', completion_window='24h', created_at=1742833922, endpoint='/v1/chat/completions', input_file_id='file-Q2nb2DDM5idxjmsQMK4v9L', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1742920322, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

### Check the batch status

In [13]:
import time

def wait_for_batch_completion(batch_id, client, interval=10):
    while True:
        batch = client.batches.retrieve(batch_id)
        print(f"Current status: {batch.status}")
        if batch.status == 'completed':
            print("Batch processing completed.")
            break
        time.sleep(interval)
    return batch

In [22]:
batch = wait_for_batch_completion(batch_creation_response.id, client)

Current status: completed
Batch processing completed.


In [23]:
model_responses = client.files.content(batch.output_file_id).text
print(model_responses)

{"id": "batch_req_67e18935d4f88190910fb7ae4a1b93dd", "custom_id": "request-0", "response": {"status_code": 200, "request_id": "4f783f2b8635e64984f8885b6fb3e221", "body": {"id": "chatcmpl-BEf0Wu4BYMWwldBRf1yvsO8uk21po", "object": "chat.completion", "created": 1742833924, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n    \"label\": \"Unsubstantiate\",\n    \"explanation\": \"The reference article does not mention anything related to reducing irreversibility or optimizing energy-consumed devices, so it does not support the statement.\"\n}", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 1188, "completion_tokens": 47, "total_tokens": 1235, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "sy

In [24]:
import json

# Parse the model_responses into a list of objects
responses_list = [json.loads(line) for line in model_responses.splitlines()]

# Print the parsed responses
print(responses_list)

[{'id': 'batch_req_67e18935d4f88190910fb7ae4a1b93dd', 'custom_id': 'request-0', 'response': {'status_code': 200, 'request_id': '4f783f2b8635e64984f8885b6fb3e221', 'body': {'id': 'chatcmpl-BEf0Wu4BYMWwldBRf1yvsO8uk21po', 'object': 'chat.completion', 'created': 1742833924, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n    "label": "Unsubstantiate",\n    "explanation": "The reference article does not mention anything related to reducing irreversibility or optimizing energy-consumed devices, so it does not support the statement."\n}', 'refusal': None, 'annotations': []}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1188, 'completion_tokens': 47, 'total_tokens': 1235, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'service_tier': 'default', 'system_fi

In [25]:
responses_dict = {int(response['custom_id'].split('-')[1]): response for response in responses_list}

In [26]:
# Create a new column in the dataframe to store the responses
if 'Model Classification' not in df.columns:
    df['Model Classification'] = None

# Iterate through the dataframe
for index, row in df.iterrows():
    if row['Reference Article Downloaded'] == 'Yes':

        model_response = responses_dict[index]['response']['body']['choices'][0]['message']['content']
        
        # Save the response to the new column
        df.at[index, 'Model Classification'] = model_response

In [27]:
df.to_pickle(f"../data/dfs/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/ReferenceErrorDetection_data_with_prompt_results_batched.pkl")

## Prompting the models (no batching)

In [34]:
# Read the content of open_ai_key.txt into a variable
with open('../open_ai_key.txt', 'r') as file:
    open_ai_key = file.read().strip()

In [35]:
from openai import OpenAI
client = OpenAI(api_key=open_ai_key)

def send_prompt(prompt, model="gpt-3.5-turbo-0125"):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0,
        timeout=30,

    )
    return completion.choices[0].message.content

In [36]:
send_prompt(example_prompt)

'{\n'

In [37]:
# path = f"../data/dfs/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/ReferenceErrorDetection_data_with_prompt_results.pkl"
# df = pd.read_pickle(path)

In [38]:
ids_to_prompt = []

In [39]:
%%time

# Create a new column in the dataframe to store the responses
if 'Model Classification' not in df.columns:
    df['Model Classification'] = None

# Iterate through the dataframe
for index, row in df.iterrows():
    if row['Reference Article Downloaded'] == 'Yes':
        if len(ids_to_prompt) != 0 and row['Reference Article ID'] not in ids_to_prompt:
            continue

        print(f"Processing: " + row['Reference Article ID'])

        # Create the prompt
        prompt = create_prompt(row)
        
        # Send the prompt and get the response
        response = send_prompt(prompt)
        
        # Save the response to the new column
        df.at[index, 'Model Classification'] = response

Processing: r001
Processing: r002
Processing: r003
Processing: r004
Processing: r005
Processing: r006
Processing: r007
Processing: r008
Processing: r009
Processing: r010
Processing: r011
Processing: r012
Processing: r013
Processing: r013
Processing: r014
Processing: r015
Processing: r005
Processing: r017
Processing: r018
Processing: r019
Processing: r020
Processing: r021
Processing: r022
Processing: r023
Processing: r024
Processing: r013
Processing: r025
Processing: r026
Processing: r027
Processing: r028
Processing: r029
Processing: r030
Processing: r031
Processing: r032
Processing: r033
Processing: r034
Processing: r035
Processing: r036
Processing: r037
Processing: r038
Processing: r039
Processing: r040
Processing: r041
Processing: r042
Processing: r043
Processing: r044
Processing: r045
Processing: r046
Processing: r047
Processing: r048
Processing: r049
Processing: r050
Processing: r051
Processing: r052
Processing: r053
Processing: r051
Processing: r055
Processing: r056
Processing: r0

In [41]:
df.to_pickle(f"../data/dfs/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/ReferenceErrorDetection_data_with_prompt_results.pkl")